The .ns5 or .nf3 files must be stored in a folder called input in the same path as this Jupyter Notebook. The results will be saved in a folder called output in the same path. 

In [1]:
import os
import numpy as np
import glob
from core.parameter_functions.Parameters import par
from scipy.io import loadmat
import multiprocessing
import sys ; 

In [2]:
from pathlib import Path

# Current notebook folder
notebook_dir = Path().resolve()  

# Add core folder
core_path = notebook_dir / "core"
sys.path.append(str(core_path))

# Add new_processing_pipeline folder
new_pipeline_path = notebook_dir.parent / "codes_emu" / "codes_for_analysis" / "new_processing_pipeline"
sys.path.append(str(new_pipeline_path))

In [4]:
from parse_ripple import parse_ripple
from core.filtering import filtering
from core.plot_continuous_bundles import PlotBundles
from core.spikeDetection import Spikes
from core.Collision import Collision
from core.ArtifactRejection import ArtifactRejection
from core.FeatureExtraction import FeatureExtractor
from core.Clustering import SPCClustering
from core.TemplateMatching import TemplateMatcher
from core.SpikeMetrics import SpikeMetricsCalculator
from core.RescueSpikes import SpikeRescuer
from core.MergeClusters import ClusterMerger

In [ ]:
# Create input and output directories
input_path = Path('input')
output_path = Path('output')

for folder in [input_path, output_path]:
    folder.mkdir(exist_ok=True)

# Check if input has data
if not any(input_path.iterdir()):
    print("Please place your raw data files in the 'input' folder")
else:
    print(f"Found {len(list(input_path.glob('*')))} file(s) in input folder")
    
print(f"\nFolders ready:")
print(f"  Input:  {input_path.resolve()}")
print(f"  Output: {output_path.resolve()}")

In [7]:
param = par()
param.parallel = True
if param.micros:
    binary_ext = '.NC5'
    ext = '.ns5'
else:
    binary_ext = '.NC3'
    ext = '.nf3'
file_paths = glob.glob("input/*"+binary_ext)
if file_paths == []:
    file_paths_ns5 = glob.glob("input/*"+ext)
    parse_ripple(file_paths_ns5)
    file_paths = glob.glob("input/*"+binary_ext)
NSx_file_path = os.path.abspath(glob.glob("input/NSx.mat")[0])
#dir_path = os.path.dirname(os.path.realpath(__file__))
dir_path = Path().resolve()
#pics_used_dir = dir_path + '/input/pics_used'
pics_used_dir = dir_path / 'input' / 'pics_used'
metadata = loadmat(NSx_file_path)
nsx = metadata['NSx']
if param.micros:
    all_channels = nsx['chan_ID'][0][list(set(np.where(nsx['unit']=='uV')[1]) & set(np.where(nsx['sr']==30000)[1]))]
else:
    all_channels = nsx['chan_ID'][0][list(set(np.where(nsx['sr']==2000)[1]))]

In [8]:
# To select and process specific channels for the rest of the pipeline, instead of all, edit the following block of code following the mentioned comments:

specific_channels = 'all'  # Use all channels
# specific_channels = [290, 291, 292, 293]  # Use specific channels
# specific_channels = 290  # Use single channel
# specific_channels = list(range(290, 301))  # Use range 290-300 (inclusive)

In [9]:
filter = filtering(save_fig=False,show_img=True,direc_resus_bae=str(Path().resolve()),
                    resus_folder_name='spectra',direc_raw=str(Path().resolve()),with_NoNotch = False,
                    time_plot_duration = 1,freq_line=60,parallel=param.parallel,k_periodograms=200,notch_filter=True,spectrum_resolution=0.5)

# To select specific channels:
# specific_channels = 'all'  # Use all channels
specific_channels = [257, 263, 264]  # Use specific channels
# specific_channels = 290  # Use single channel
# specific_channels = list(range(290, 301))  # Use range 290-300 (inclusive)

filter.new_check_lfp_power_NSX(metadata, channels = specific_channels)

Processing 3 channels: [array([257], dtype=object), array([263], dtype=object), array([264], dtype=object)]


/media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/core/filtering.py:311: RuntimeWarning: divide by zero encountered in log
  flog_data1 = np.log(f)



Processing complete! Results saved to /media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/output/process_info.mat


In [ ]:
plt = PlotBundles()

# To select specific channels:
specific_channels = 'all'  # Use all channels
# specific_channels = [257, 263]  # Use specific channels
# specific_channels = 290  # Use single channel
# specific_channels = list(range(290, 301))  # Use range 290-300 (inclusive)

plt.plot(nsx_file=nsx, par=param, channels=specific_channels, notchfilter=1)

Plotting all 68 channels
Plotting bundle mLAMY with 8 channels: [array([[266]], dtype=uint16) array([[267]], dtype=uint16)
 array([[268]], dtype=uint16) array([[269]], dtype=uint16)
 array([[270]], dtype=uint16) array([[271]], dtype=uint16)
 array([[272]], dtype=uint16) array([[273]], dtype=uint16)]
Saved: z:\Tests\Matlab sorting pipeline\Tapasi\MCWs_Pipeline\full_processing_pipeline/output/mLAMY_4_filtorder1withnothesneg.png
Plotting bundle mLFUS with 8 channels: [array([[289]], dtype=uint16) array([[290]], dtype=uint16)
 array([[291]], dtype=uint16) array([[292]], dtype=uint16)
 array([[293]], dtype=uint16) array([[294]], dtype=uint16)
 array([[295]], dtype=uint16) array([[296]], dtype=uint16)]
Saved: z:\Tests\Matlab sorting pipeline\Tapasi\MCWs_Pipeline\full_processing_pipeline/output/mLFUS_4_filtorder1withnothesneg.png
Plotting bundle mLHIPH with 8 channels: [array([[298]], dtype=uint16) array([[299]], dtype=uint16)
 array([[300]], dtype=uint16) array([[301]], dtype=uint16)
 array(

In [11]:
# Get all available channels
if param.micros:
    all_channels = nsx['chan_ID'][0][list(set(np.where(nsx['unit']=='uV')[1]) & 
                                          set(np.where(nsx['sr']==30000)[1]))]
else:
    all_channels = nsx['chan_ID'][0][list(set(np.where(nsx['sr']==2000)[1]))]

# ========================================
# CHANNEL SELECTION (NON-INTERACTIVE)
# ========================================

# Option 1: Use all channels
# specific_channels = 'all'

# Option 2: Use specific channels
specific_channels = [257, 263, 264]

# Option 3: Use single channel
# specific_channels = 290

# Option 4: Use range of channels
# specific_channels = list(range(290, 301))  # 290-300

# ========================================
# DETECTION TYPE ASSIGNMENT
# ========================================

# Option 1: All channels use negative detection (default)
neg_channels = 'all'
pos_channels = None

# Option 2: Specific channels for each detection type
# neg_channels = [290, 291, 292]
# pos_channels = [293, 294]
# # Remaining channels will use 'both' detection

# Option 3: All use negative, none use positive
# neg_channels = 'all'
# pos_channels = None

# ========================================
# PROCESS CHANNEL SELECTION
# ========================================

def parse_channels(channel_spec, all_channels):
    """Convert channel specification to channel objects."""
    if isinstance(channel_spec, str) and channel_spec.lower() == 'all':
        return all_channels
    elif channel_spec is None or (isinstance(channel_spec, list) and len(channel_spec) == 0):
        return np.array([], dtype=object)
    else:
        if isinstance(channel_spec, int):
            channel_spec = [channel_spec]
        return np.array([ch for ch in all_channels if ch[0] in channel_spec], dtype=object)

# Parse specific_channels
if isinstance(specific_channels, str) and specific_channels.lower() == 'all':
    channels = all_channels
    print(f"Processing all {len(channels)} channels: {[ch[0] for ch in channels]}")
else:
    if isinstance(specific_channels, int):
        specific_channels = [specific_channels]
    channels = np.array([ch for ch in all_channels if ch[0] in specific_channels], dtype=object)
    print(f"Processing {len(channels)} specific channels: {[ch[0] for ch in channels]}")

# Parse neg_channels and pos_channels
neg_thr_channels = parse_channels(neg_channels, channels)
pos_thr_channels = parse_channels(pos_channels, channels)

print(f"neg_thr_channels: {[ch[0] for ch in neg_thr_channels] if len(neg_thr_channels) > 0 else 'None'}")
print(f"pos_thr_channels: {[ch[0] for ch in pos_thr_channels] if len(pos_thr_channels) > 0 else 'None'}")

# ========================================
# SETUP PARAMETERS
# ========================================

del param
param = par()
param.detection = 'neg'
param.sr = 30000
param.detect_fmin = 300
param.detect_fmax = 3000
param.auto = 0
param.mVmin = 50
param.w_pre = 20                       
param.w_post = 44                     
param.min_ref_per = 1.5                                    
param.ref = np.floor(param.min_ref_per * param.sr / 1000)                  
param.ref = param.ref
param.factor_thr = 5
param.detect_order = 4
param.sort_order = 2
param.detect_fmin = 300
param.sort_fmin = 300
param.stdmin = 5
param.stdmax = 50
param.ref_ms = 1.5
param.preprocessing = True
param.minus_one = 0
param.interpolation = 'n'
param.segments_length = 5
param.tmax = 'all'
param.cont_plot_samples = 60000

# ========================================
# RUN SPIKE DETECTION
# ========================================

print('\nStarting spike detection...')
param.detection = 'neg'

if param.parallel:
    spike = Spikes(par=param, nsx=nsx)
    
    # Process negative detection channels
    if neg_thr_channels.size:
        print(f"Processing {len(neg_thr_channels)} channels with negative detection...")
        with multiprocessing.Pool(processes=10) as pool:
            pool.map(spike.get_spikes, neg_thr_channels, chunksize=1)
            pool.close()
            pool.join()
    
    # Process positive detection channels
    param.detection = 'pos'
    if pos_thr_channels.size:
        print(f"Processing {len(pos_thr_channels)} channels with positive detection...")
        with multiprocessing.Pool(processes=10) as pool:
            pool.map(spike.get_spikes, pos_thr_channels, chunksize=1)
            pool.close()
            pool.join()
    
    # Process 'both' detection channels
    param.detection = 'both'
    both_thr_channels = np.setdiff1d(
        np.setdiff1d(
            np.array([ch[0] for ch in channels]),
            np.array([ch[0] for ch in neg_thr_channels])
        ),
        np.array([ch[0] for ch in pos_thr_channels]) if pos_thr_channels.size else []
    )
    
    if both_thr_channels.size:
        # Convert back to channel objects
        both_thr_channels = np.array([ch for ch in channels if ch[0] in both_thr_channels], dtype=object)
        print(f"Processing {len(both_thr_channels)} channels with both detection...")
        with multiprocessing.Pool(processes=10) as pool:
            pool.map(spike.get_spikes, both_thr_channels, chunksize=1)
            pool.close()
            pool.join()
    
    print('Spike detection done!')
    
else:
    spike = Spikes(par=param, nsx=nsx)
    
    # Process negative detection channels
    if neg_thr_channels.size:
        print(f"Processing {len(neg_thr_channels)} channels with negative detection...")
        for channel in neg_thr_channels:
            spike.get_spikes(channel)
    
    # Process positive detection channels
    param.detection = 'pos'
    if pos_thr_channels.size:
        print(f"Processing {len(pos_thr_channels)} channels with positive detection...")
        for channel in pos_thr_channels:
            spike.get_spikes(channel)
    
    # Process 'both' detection channels
    param.detection = 'both'
    both_thr_channels = np.setdiff1d(
        np.setdiff1d(
            np.array([ch[0] for ch in channels]),
            np.array([ch[0] for ch in neg_thr_channels])
        ),
        np.array([ch[0] for ch in pos_thr_channels]) if pos_thr_channels.size else []
    )
    
    if both_thr_channels.size:
        # Convert back to channel objects
        both_thr_channels = np.array([ch for ch in channels if ch[0] in both_thr_channels], dtype=object)
        print(f"Processing {len(both_thr_channels)} channels with both detection...")
        for channel in both_thr_channels:
            spike.get_spikes(channel)
    
    print('Spike detection done!')

Processing 3 specific channels: [array([257], dtype=object), array([263], dtype=object), array([264], dtype=object)]
neg_thr_channels: [array([257], dtype=object), array([263], dtype=object), array([264], dtype=object)]
pos_thr_channels: None

Starting spike detection...
Processing 3 channels with negative detection...
Spike file already exists for channel 264, skipping...
Spike file already exists for channel 263, skipping...
Spike file already exists for channel 257, skipping...
Spike detection done!


In [6]:
col = Collision(all_channels,nsx)

# To select specific channels:
specific_channels = 'all'  # Use all channels
# specific_channels = [257, 263]  # Use specific channels
# specific_channels = 290  # Use single channel
# specific_channels = list(range(290, 301))  # Use range 290-300 (inclusive)

col.separate_collisions(channels=specific_channels)

Processing collisions for all 68 channels: [array([257], dtype=uint16), array([258], dtype=uint16), array([259], dtype=uint16), array([260], dtype=uint16), array([261], dtype=uint16), array([262], dtype=uint16), array([263], dtype=uint16), array([264], dtype=uint16), array([266], dtype=uint16), array([267], dtype=uint16), array([268], dtype=uint16), array([269], dtype=uint16), array([270], dtype=uint16), array([271], dtype=uint16), array([272], dtype=uint16), array([273], dtype=uint16), array([289], dtype=uint16), array([290], dtype=uint16), array([291], dtype=uint16), array([292], dtype=uint16), array([293], dtype=uint16), array([294], dtype=uint16), array([295], dtype=uint16), array([296], dtype=uint16), array([298], dtype=uint16), array([299], dtype=uint16), array([300], dtype=uint16), array([301], dtype=uint16), array([302], dtype=uint16), array([303], dtype=uint16), array([304], dtype=uint16), array([305], dtype=uint16), array([321], dtype=uint16), array([322], dtype=uint16), arra

In [8]:
# Initialize
artifact_rej = ArtifactRejection(
    input_dir=str(Path().resolve() / 'output'),
    output_dir=None,  # None = same as input_dir
    backup_dir=None   # None = input_dir/backup
)

# Set thresholds (optional - defaults shown)
artifact_rej.set_thresholds(
    prom_ratio=2.0,           # Prominence ratio threshold
    width_upper=15.0,         # Upper width limit
    width_lower=3.0,          # Lower width limit
    low_amp_percentile=5.0,   # Low amplitude percentile
    other_prom_min_ratio=0.01 # 1% rule for other prominence
)

# ========================================
# CHANNEL SELECTION
# ========================================

# To select specific channels:
# specific_channels = 'all'  # Use all channels
specific_channels = [257, 263]  # Use specific channels
# specific_channels = 290  # Use single channel
# specific_channels = list(range(290, 301))  # Use range 290-300 (inclusive)

# ========================================
# PROCESS ARTIFACT REJECTION
# ========================================

artifact_rej.process_all_channels(channels=specific_channels, parallel=True)

Selected channels: [257, 263]
Found 2 channels to process
Thresholds: Prom>2.0, Width>3.0 & <15.0
Low Amp Percentile: 5.0%, Other Prom Min: 1.0%


Processing channel 257: mLTP01 raw_257_spikes.mat
Loaded 24007 spikes (using spikes)
Calculating metrics for 24007 spikes...
  Using 80 CPU cores for parallel processing...
Applying artifact rejection logic...
  Amplitude threshold (5.0th percentile): 22.06
  Auto-quarantined (bad ratio/width): 81
  Low amplitude: 1201
  One-peak exceptions preserved: 23

Results:
  Total spikes: 24007
  Preserved: 19455 (81.0%)
  Quarantined: 4462 (18.6%)
  Bundle collision: 296 (1.2%)
  Both quarantined & bundle: 206
Saved updated file to: /media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/output/mLTP01 raw_257_spikes.mat

Processing channel 263: mLTP07 raw_263_spikes.mat
Loaded 7425 spikes (using spikes)
Calculating metrics for 7425 spikes...
  Using 80 CPU cores for parallel processing...
Applying artifact reject

In [22]:
# Define input/output directories
input_dir = str(Path().resolve() / 'output')
output_dir = str(Path().resolve() / 'output' / 'features')

# GMM (recommended)
extractor = FeatureExtractor(input_dir, output_dir, method='gmm')
#extractor.process_all_channels(channels='all')
extractor.process_all_channels(channels=[304])

# # Haar (fast)
# extractor = FeatureExtractor(input_dir, output_dir, method='haar')
# extractor.process_all_channels(channels=[257, 263])

# # PCA
# extractor = FeatureExtraction(input_dir, output_dir, method='pca', n_components=10)
# extractor.process_all_channels(channels=list(range(290, 301)))


######################################################################
FEATURE EXTRACTION PIPELINE
######################################################################
Method: GMM
Input directory: /media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/output
Output directory: /media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/output/features
Channels: [304]
Total channels: 1
Scales: 4, Min weight: 0.005, Min coeff: 5
######################################################################

[1/1] Processing channel 304...

Channel 304 - GMM extraction
Loaded 2097 spikes (shape: (2097, 64))
  Running GMM_1channel (scales=4)...
  Processed 64 coefficients
  Filtering components (min_weight=0.005)...
  Expanding metrics into vectors...
  Applying knee detection...
  Selecting final coefficients...
  Selected 10 coefficients
Saved features to: ch304_features_gmm.mat
Feature matrix shape: (2097, 10)

###########

In [8]:
# Cluster all channels in feature directory
clusterer = SPCClustering(
    input_dir=str(Path().resolve() / 'output' / 'features'),
    times_output_dir=str(Path().resolve() / 'output'),  # Explicitly set output dir
    min_clus=20,
    temperature=0.15,
    knn=11,
    generate_times_files=True,  # Explicitly enable
    verbose=True  # Show all logs
)
#clusterer.process_all_channels(channels='all')

# Cluster specific channels
clusterer.process_all_channels(channels=[328])


######################################################################
SPC CLUSTERING PIPELINE
######################################################################
Input directory: /media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/output/features
Output directory: /media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/output/features
Times output directory: /media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/output
Channels: [328]
Total channels: 1
Parameters:
  Min cluster size: 20
  Max clusters: 20
  Temperature: 0.15
  K-nearest neighbors: 11
  Generate times files: True
  Method: Fallback SPC
######################################################################

[1/1] Processing channel 328...

Channel 328 - SPC Clustering
File: ch328_features_gmm.mat (feature file)
Loaded 82 spikes with 10 features
  Computing pairwise distances...
  Finding 11 nearest nei

In [4]:
# Initialize the matcher
matcher = TemplateMatcher(
                    input_dir=str(Path().resolve() / 'output'),
                    template_sdnum=3.0,
                    metric='euclidean'
                )
                
## Match unclassified spikes in ALL channels
# matcher.process_all_channels(channels='all')
                
# Match unclassified spikes in specific channels
matcher.process_all_channels(channels=[328])
                


######################################################################
TEMPLATE MATCHING PIPELINE
######################################################################
Input directory: /media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/output
Output directory: /media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/output
Total channels to process: 1
Parameters: {'template_sdnum': 3.0, 'template_k': None, 'template_k_min': 1, 'use_pointdist': False, 'pointlimit': None, 'metric': 'euclidean'}
######################################################################


Channel 328 - Template Matching
File: times_ch328.mat
Skipping: No classified spikes found to build templates.

######################################################################
TEMPLATE MATCHING COMPLETE
######################################################################
Successful: 1/1
Failed: 0/1
Output directory: /media/sEEG_DATA/Tests

In [ ]:
metrics_calc = SpikeMetricsCalculator(
        input_dir=str(Path().resolve() / 'output'),
        output_dir=str(Path().resolve() / 'output/Quality_Metrics')
    )
    
## Run on all channels
#metrics_calc.process_all_channels(channels='all')
    
## Run on specific channels
metrics_calc.process_all_channels(channels=[328])

In [ ]:
# 2. Define the directories
# This assumes your notebook is in 'full_processing_pipeline'
base_dir = Path().resolve() 
times_dir = str(base_dir / 'output')
spikes_dir = str(base_dir / 'output')

# 3. Initialize the SpikeRescuer
# You can choose 'euclidean' or 'correlation'
rescuer = SpikeRescuer(
    times_dir=times_dir,
    spikes_dir=spikes_dir,
    output_dir=times_dir,  # This will update the times_*.mat files in-place
    template_sdnum=3.0,    # Use 3.0 for 'euclidean'
    metric='euclidean'
)

# --- 4. Run the rescue operation ---

# Option A: Run on ALL channels
# This will find all channels that have both a 'times_*.mat'
# and a '*_spikes.mat' file and process them.
# print("--- Running Spike Rescue on ALL channels ---")
#rescuer.process_all_channels(channels='all')


# Option B: Run on specific channels
print("\n--- Running Spike Rescue on specific channels ---")
specific_channels = [257]
rescuer.process_all_channels(channels=specific_channels)

In [5]:
# Initialize the ClusterMerger
merger = ClusterMerger(
    input_dir=str(Path().resolve() / 'output'),
    output_dir=str(Path().resolve() / 'output_merged_mat'),
    report_dir=str(Path().resolve() / 'output_merged_reports'),
    calc_metrics=True,
    verbose=True,
    clusters_per_page=6  # Optional: kwargs for make_cluster_report
)

# Define merge groups (optional)
# Each sublist: [target_cluster, source_cluster1, source_cluster2, ...]
merge_rules = [
    [1, 3],    # Merge cluster 3 into cluster 1
    [2, 5, 7]  # Merge clusters 5 and 7 into cluster 2
]

# Run on all channels
# merger.process_all_channels(channels='all', merge_groups=merge_rules)

# Run on specific channels with merging
merger.process_all_channels(channels=[328], merge_groups=merge_rules)

# Or run without merging (just renumber clusters)
# merger.process_all_channels(channels=[328], merge_groups=None)

Created directory: /media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/output_merged_mat
Created directory: /media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/output_merged_reports

######################################################################
CLUSTER MERGING & REPORTING PIPELINE
######################################################################
Input directory: /media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/output
Output MAT directory: /media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/output_merged_mat
Output PDF directory: /media/sEEG_DATA/Tests/Matlab sorting pipeline/Tapasi/MCWs_Pipeline/full_processing_pipeline/output_merged_reports
Total channels to process: 1
Merge Groups: [[1, 3], [2, 5, 7]]
######################################################################


Channel 328 - Cluster Merging
Fil